In [53]:
import numpy as np
import pandas as pd
import nltk
# nltk.download("punkt")
import re
import networkx as nx

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
df = pd.read_csv("../../files/cgu_figaro.csv", sep="|", header=None, names=["id", "text", "source"])
df.head()

,id,text,source
0,1,Conditions Générales d'Utilisation\n\n \n\nSoc...,http://mentions-legales.lefigaro.fr/page/cgu


In [16]:
sentences = []
for s in df["text"]:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [17]:
sentences[:5]

["Conditions Générales d'Utilisation\n\n \n\nSociété du Figaro, SAS au capital de 16.860.475 €, dont le siège est 14, bd Haussmann, 75009 PARIS, immatriculée au Registre du Commerce et des Sociétés de Paris sous le numéro 542 077 755 (ci-après « Le Figaro »), édite un site d'informations générales www.lefigaro.fr, ci-après le « Site » permettant un accès à différentes informations, ainsi qu'à différents services et produits complémentaires, détaillés ci-après.",
 'Les présentes conditions générales d\'utilisation, ("CGU"), soumises au droit français, ont vocation à régir l\'utilisation du Site ainsi que l\'utilisation des différents services payants ou gratuits (ci-après le(s) « Service(s) »).',
 "La navigation dans le Site et/ou l'inscription à un Service emporte de plein droit acceptation expresse et sans réserve des CGU par l'utilisateur du Site.",
 "En cas de non respect des termes des présentes CGU, Le Figaro se réserve le droit de prendre toute mesure de nature à préserver ses in

In [21]:
word_embeddings = {}
f = open('ressources/glove.6B.100d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

400000

In [23]:
word_embeddings[list(word_embeddings)[0]]

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [29]:
clean_sentences = []
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Zàéêèç]" , " ")
clean_sentences = [sentence.lower() for sentence in clean_sentences]
clean_sentences[:5]

['conditions générales d utilisation     société du figaro  sas au capital de               dont le siège est     bd haussmann        paris  immatriculée au registre du commerce et des sociétés de paris sous le numéro              ci après   le figaro     édite un site d informations générales www lefigaro fr  ci après le   site   permettant un accès à différentes informations  ainsi qu à différents services et produits complémentaires  détaillés ci après ',
 'les présentes conditions générales d utilisation    cgu    soumises au droit français  ont vocation à régir l utilisation du site ainsi que l utilisation des différents services payants ou gratuits  ci après le s    service s     ',
 'la navigation dans le site et ou l inscription à un service emporte de plein droit acceptation expresse et sans réserve des cgu par l utilisateur du site ',
 'en cas de non respect des termes des présentes cgu  le figaro se réserve le droit de prendre toute mesure de nature à préserver ses intérêts 

In [34]:
stop_words = stopwords.words('french')
type(stop_words)
stop_words[:5]

['au', 'aux', 'avec', 'ce', 'ces']

In [35]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences[:5]

['conditions générales utilisation société figaro sas capital dont siège bd haussmann paris immatriculée registre commerce sociétés paris sous numéro ci après figaro édite site informations générales www lefigaro fr ci après site permettant accès différentes informations ainsi différents services produits complémentaires détaillés ci après',
 'présentes conditions générales utilisation cgu soumises droit français vocation régir utilisation site ainsi utilisation différents services payants gratuits ci après service',
 'navigation site inscription service emporte plein droit acceptation expresse sans réserve cgu utilisateur site',
 'cas non respect termes présentes cgu figaro réserve droit prendre toute mesure nature préserver intérêts notamment assurer exécution',
 'figaro pourra notamment cette fin assigner utilisateur site responsabilité civile pénale']

In [45]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split()) + 0.001)
    else:
        v =np.zeros((100,))
    sentence_vectors.append(v)
sentence_vectors[:1]

[array([-0.043921  , -0.15585803, -0.06155535,  0.12787385, -0.10662362,
         0.0469764 ,  0.17377853,  0.24399895,  0.16045878,  0.00308373,
         0.11150715,  0.04853465,  0.00824986, -0.07552316, -0.03351381,
        -0.0236734 ,  0.09404748, -0.20686445, -0.10770278,  0.08203354,
         0.05633522,  0.03490487, -0.25748604,  0.03927284,  0.0252534 ,
        -0.19922536,  0.0673274 ,  0.00629042,  0.00092614, -0.11780562,
        -0.2075894 , -0.19914777, -0.09256971, -0.02773196,  0.01562619,
         0.20747097,  0.18080597,  0.00786068, -0.14068395, -0.01582064,
         0.13102771, -0.01898534, -0.28176171, -0.06420881,  0.05535709,
        -0.10612473, -0.30782937, -0.13849781,  0.0061038 ,  0.00425504,
        -0.01695286,  0.24848781, -0.05175791,  0.01179037, -0.34548978,
        -0.30691616,  0.21950321,  0.23483718,  0.0998589 ,  0.00550106,
        -0.10996187, -0.13990714, -0.21502218, -0.11379474,  0.04040608,
         0.08793994, -0.18039169, -0.18588398,  0.2

In [52]:
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1, 100))[0,0]
sim_mat[1, 2]

0.6065083852529346

In [56]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
scores

{0: 0.008451211884302591,
 1: 0.007117768139346594,
 2: 0.0072046569590507184,
 3: 0.007624377623785239,
 4: 0.007983388338201421,
 5: 0.007232456639807979,
 6: 0.006959648380572799,
 7: 0.008709778176525452,
 8: 0.006816780457300552,
 9: 0.008145876495118757,
 10: 0.007584994983858993,
 11: 0.006033130973915544,
 12: 0.00801067484837502,
 13: 0.008180603217977834,
 14: 0.00712842323265923,
 15: 0.00691568466342301,
 16: 0.007530660753356505,
 17: 0.004029706227955555,
 18: 0.007650105346906994,
 19: 0.006436423293714076,
 20: 0.007768614132240583,
 21: 0.007480457585159912,
 22: 0.007027205553795279,
 23: 0.00540080642903481,
 24: 0.008358806764460654,
 25: 0.006493867963071031,
 26: 0.00817359751832969,
 27: 0.007215135374098565,
 28: 0.0031257769226417167,
 29: 0.006963109224982416,
 30: 0.004670429168500729,
 31: 0.006924009505735603,
 32: 0.008254567107636982,
 33: 0.005407174657294,
 34: 0.007279529590009251,
 35: 0.008095327759682682,
 36: 0.007248830709840632,
 37: 0.0076643803

In [58]:
resultat = ""
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
for i in range(10):
    resultat += ranked_sentences[i][1]

In [59]:
print(resultat)

Etant de fait soumis à une obligation de moyens, Le Figaro ne saurait être tenu responsable de tout dommage, quelle qu'en soit la nature, résultant d'une indisponibilité du Site ou d'un ou de plusieurs Services, ou résultant de la présence de virus dans le Site.De même, les informations que l'Utilisateur obtient dans le Site ou dans le cadre des Services proposés par Le Figaro, que ce soit dans le Site, par courrier électronique ou sous forme de message texte, ne donnent droit à aucune garantie du Figaro.Article 1 - Définitions

Compte : désigne l'espace mis à la disposition de l'Utilisateur inscrit sur le Site, lui permettant d'accéder, après s'être identifié par ses Codes d'accès, aux Services souscrits, ainsi qu’à son contrat et factures éventuelles.L’Utilisateur peut demander à tout moment que ses données personnelles, collectées dans le cadre de l’utilisation du Service Figaro Immobilier, ne soient pas communiquées et dispose à cette fin d’un d'un droit d'accès, d’opposition et de

In [61]:
f = open("resultat/resultat_summarization.txt", "w")
f.write(resultat)
f.close()

In [62]:
print(ranked_sentences)

[(0.008806335959100926, "Etant de fait soumis à une obligation de moyens, Le Figaro ne saurait être tenu responsable de tout dommage, quelle qu'en soit la nature, résultant d'une indisponibilité du Site ou d'un ou de plusieurs Services, ou résultant de la présence de virus dans le Site."), (0.00874356772744328, "De même, les informations que l'Utilisateur obtient dans le Site ou dans le cadre des Services proposés par Le Figaro, que ce soit dans le Site, par courrier électronique ou sous forme de message texte, ne donnent droit à aucune garantie du Figaro."), (0.008709778176525452, "Article 1 - Définitions\n\nCompte : désigne l'espace mis à la disposition de l'Utilisateur inscrit sur le Site, lui permettant d'accéder, après s'être identifié par ses Codes d'accès, aux Services souscrits, ainsi qu’à son contrat et factures éventuelles."), (0.00852576085370038, "L’Utilisateur peut demander à tout moment que ses données personnelles, collectées dans le cadre de l’utilisation du Service Fig

In [15]:
with open('ressources/glove.6B.100d.txt', encoding='utf8') as myfile:
    [print(next(myfile)) for x in range(40)]
    print("\n\n\n")

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

, -0.10767 0.11053 0.59812 -0.54361 0.67396 0.10663 0.038867 0.35481 0.06351 -0.094189 0.15786 -0.81665 0.14172 0.21939 0.58505 -0.52158

In [17]:
import numpy as np

toto = {}
word="test"
coefs = np.asarray([0.153, 0.185, 0,213], dtype='float32')
toto[word] = coefs

In [18]:
toto[list(toto)[0]]

array([1.53e-01, 1.85e-01, 0.00e+00, 2.13e+02], dtype=float32)

In [23]:
toto.get("toto", np.zeros((100,)))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])